## Problem Definition

In the following different ways of loading or implementing an optimization problem in our framework are discussed.

### By Class

A very detailed description of defining a problem through a class is already provided in the [Getting Started Guide](../getting_started.ipynb).
In the following the definition of a simple optimization problem with **one** objective and **two** constraints is considered. The problem has two constants, *const_1* and *const_2* which can be modified by initiating the problem with different parameters. By default, it consists of 10 variable and the lower and upper bounds are within $[-5, 5]$ for all variables. 

**Note**: The example below uses the `autograd` library which allows to calculate the gradients through automatic differentiation.

In [23]:
import autograd.numpy as anp

from pymoo.model.problem import Problem

class MyProblem(Problem):

    def __init__(self, const_1=5, const_2=0.1):

        # define lower and upper bounds -  1d array with length equal to number of variable
        xl = -5 * anp.ones(10)
        xu = 5 * anp.ones(10)

        super().__init__(n_var=10, n_obj=1, n_constr=2, xl=xl, xu=xu, evaluation_of="auto")

        # store custom variables needed for evaluation
        self.const_1 = const_1
        self.const_2 = const_2

    def _evaluate(self, x, out, *args, **kwargs):
        f = anp.sum(anp.power(x, 2) - self.const_1 * anp.cos(2 * anp.pi * x), axis=1)
        g1 = (x[:, 0] + x[:, 1]) - self.const_2
        g2 = self.const_2 - (x[:, 2] + x[:, 3])

        out["F"] = f
        out["G"] = anp.column_stack([g1, g2])



After creating a problem object the evaluate function can be called. The *return_values_of* variable can be overwritten to modify the list of returned parameters. The gradients for the objectives `dF` and constraints `dG` can be obtained as follows:

In [25]:
problem = MyProblem()
F, G, CV, feasible, dF, dG = problem.evaluate(np.random.rand(100, 10),
                                              return_values_of=["F", "G", "CV", "feasible", "dF", "dG"])


**Elementwise Evaluation**

If the problem can not be executed by using matrix operations a serialized evaluation can be indicated by using the `elementwise_evaluation=True` flag. If the flag is set then an outer loop is already implemented an `x` is only an **one** dimensional array.

In [34]:
class MyProblem(Problem):

    def __init__(self, **kwargs):
        super().__init__(n_var=2, n_obj=1, elementwise_evaluation=True, **kwargs)

    def _evaluate(self, x, out, *args, **kwargs):
        out["F"] = x.sum()

### By Function

Another way of defining a problem is through functions. One the one hand many function calls need to be performed to evaluated a set of solutions, but on the other hand it is a very intuitive way of defining a problem.

In [32]:
import numpy as np
from pymoo.model.problem import FunctionalProblem


objs = [
    lambda x: np.sum((x - 2) ** 2),
    lambda x: np.sum((x + 2) ** 2)
]

constr_ieq = [
    lambda x: np.sum((x - 1) ** 2)
]


problem = FunctionalProblem(10,
                            objs,
                            constr_ieq=constr_ieq,
                            xl=np.array([-10, -5, -10]),
                            xu=np.array([10, 5, 10])
                            )

F, CV = problem.evaluate(np.random.rand(3, 10))

print(f"F: {F}\n")
print(f"CV: {CV}")

F: [[21.42584785 66.09631176]
 [26.97585042 57.6450503 ]
 [21.50184569 65.41885629]]

CV: [[2.59346383]
 [4.64315039]
 [2.48109834]]


### By String

In our framework various test problems are already implemented and available by providing the corresponding problem name we have assigned to it. A couple of problems can be further parameterized by provided for instance the number of variables, constraints or other problem dependent constants.

In [27]:
from pymoo.factory import get_problem

p = get_problem("dtlz1_-1", n_var=20, n_obj=5)

# create a simple test problem from string
p = get_problem("Ackley")

# the input name is not case sensitive
p = get_problem("ackley")

# also input parameter can be provided directly
p = get_problem("dtlz1_-1", n_var=20, n_obj=5)

## API